In [1]:
fastq_folder = "/home/laura/projects/virus-watch-data/alex_ismagilov/VSP"
u_tax_csv = "/home/laura/projects/virus-watch-data/virus_ref/u_tax_nodup_clu.csv"

In [2]:
import kb_python.utils as kb_utils
import numpy as np
import pandas as pd
import json
import os
import glob
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'

# Path to Krona tool
ktImportText = "/home/laura/bin/ktImportText"

def nd(arr):
    """
    Function to transform numpy matrix to nd array.
    """
    return np.asarray(arr).reshape(-1)

Get fastq names:

In [3]:
fastqs = []
for filename in glob.glob(f"{fastq_folder}/*.fastq.gz"):
    fastqs.append(filename.split("/")[-1])
    
fastqs.sort()
fastqs

['M006457-VSP-CIT-1-Rep1_S1_L001_R1_001.fastq.gz',
 'M006457-VSP-CIT-1-Rep1_S1_L001_R2_001.fastq.gz',
 'M006457-VSP-CIT-1-Rep2_S2_L001_R1_001.fastq.gz',
 'M006457-VSP-CIT-1-Rep2_S2_L001_R2_001.fastq.gz',
 'M006457-VSP-CIT-10-Rep1_S9_L001_R1_001.fastq.gz',
 'M006457-VSP-CIT-10-Rep1_S9_L001_R2_001.fastq.gz',
 'M006457-VSP-CIT-10-Rep2_S10_L001_R1_001.fastq.gz',
 'M006457-VSP-CIT-10-Rep2_S10_L001_R2_001.fastq.gz',
 'M006457-VSP-CIT-11-Rep1_S15_L001_R1_001.fastq.gz',
 'M006457-VSP-CIT-11-Rep1_S15_L001_R2_001.fastq.gz',
 'M006457-VSP-CIT-11-Rep2_S16_L001_R1_001.fastq.gz',
 'M006457-VSP-CIT-11-Rep2_S16_L001_R2_001.fastq.gz',
 'M006457-VSP-CIT-12-Rep1_S21_L001_R1_001.fastq.gz',
 'M006457-VSP-CIT-12-Rep1_S21_L001_R2_001.fastq.gz',
 'M006457-VSP-CIT-12-Rep2_S22_L001_R1_001.fastq.gz',
 'M006457-VSP-CIT-12-Rep2_S22_L001_R2_001.fastq.gz',
 'M006457-VSP-CIT-13-Rep1_S27_L001_R1_001.fastq.gz',
 'M006457-VSP-CIT-13-Rep1_S27_L001_R2_001.fastq.gz',
 'M006457-VSP-CIT-13-Rep2_S28_L001_R1_001.fastq.gz',
 'M

Create Krona plots:

In [4]:
count_threshold = 1

In [5]:
# for fastq in fastqs:
#     # Load data
#     sample = fastq.split(".fastq.gz")[0]
#     results_dir = f"{fastq_folder}/results_full_index/{sample}"

#     # Filepath to counts
#     X = f"{results_dir}/bustools_count/output.mtx"
#     # Filepath to barcode metadata
#     var_path = f"{results_dir}/bustools_count/output.genes.txt"
#     # Filepath to gene metadata
#     obs_path = f"{results_dir}/bustools_count/output.barcodes.txt" 

#     # Create AnnData object
#     adata = kb_utils.import_matrix_as_anndata(X, obs_path, var_path)

#     ## Get virus ID -> phylogeny mapping from tsv
#     # Get corresponding phylogenies for each virus ID from look-up file
#     phylogeny_data = pd.read_csv(u_tax_tsv, sep='\t')

#     # Add total number of counts (across all cells) for each virus ID
#     virus_ids = adata.var.index.values
#     total_counts = nd(adata.X.sum(axis=0))
#     total_count_dict = {virus_ids[i]: total_counts[i] for i in range(len(virus_ids))}
#     phylogeny_data['total_count'] = phylogeny_data['Label'].map(total_count_dict)

#     # Remove all viruses that were only detected in less than num_cells_threshold cells
#     phylogeny_data_clean = phylogeny_data[phylogeny_data["total_count"] >= count_threshold]

#     # Remove non-relevent columns and change order of columns  
#     phylogeny_data_clean = phylogeny_data_clean[["total_count", "phylum", "class", "order", "family", "genus", "species", "Label"]]

#     # Replace dots with NaN
#     phylogeny_data_clean = phylogeny_data_clean.replace(".", np.nan)

#     # Save counts + taxnomomies data to txt
#     phylogeny_data.to_csv(f'{results_dir}/counts_tax.csv', index=False)
#     phylogeny_data_clean.to_csv(f'{results_dir}/krona.txt', sep ='\t', header=None, index=False)

#     # Generate Krona plot and name after sample
#     !cd $results_dir && $ktImportText -n "Viral RNAs" $results_dir/krona.txt

#     print("\n", sample)

#     # Get number of reads processed
#     with open(f'{results_dir}/run_info.json') as f:
#         data = f.read()
#     run_info_json = json.loads(data)
#     f.close()

#     # Get multimapping loss
#     mm_loss = (run_info_json["n_pseudoaligned"] - total_counts.sum()).astype(int)
#     mm_perc = np.round(((np.array(mm_loss)/np.array(run_info_json["n_pseudoaligned"])) * 100), 2)
#     print(f"Reads lost to multimapping: {mm_loss} out of {run_info_json['n_pseudoaligned']} ({mm_perc} %)")

#     # Get number of cardinality clashes
#     n_clashes = run_info_json["n_frame_clashes"]
#     c_clash_perc = np.round(((n_clashes/np.array(run_info_json['n_pseudoaligned'])) * 100), 2)
#     print(f"Ocurrences of equal cardinality of equivalence classes: {n_clashes} over {run_info_json['n_pseudoaligned']} aligned frames ({c_clash_perc} %)")

#     # Save general info in csv
#     info_df = pd.DataFrame()
#     info_df["n_reads_processed"] = [run_info_json["n_processed"]]
#     info_df["n_reads_aligned"] = [run_info_json["n_pseudoaligned"]]
#     info_df["n_reads_multimapped"] = [mm_loss]
#     info_df["mm_loss_perc"] = [mm_perc]
#     info_df["n_cardinality_clashes"] = [n_clashes]
#     info_df["cardinality_clash_perc"] = [c_clash_perc]

#     info_df.to_csv(f'{results_dir}/run_info.csv', index=False)

Create adata objects from raw data:

In [5]:
adatas = []
for fastq in fastqs:
    # Load data
    sample = fastq.split(".fastq.gz")[0]
    results_dir = f"{fastq_folder}/results_cdna_dna/{sample}"

    # Filepath to counts
    X = f"{results_dir}/bustools_count/output.mtx"
    # Filepath to barcode metadata
    var_path = f"{results_dir}/bustools_count/output.genes.txt"
    # Filepath to gene metadata
    obs_path = f"{results_dir}/bustools_count/output.barcodes.txt" 

    # Create AnnData object
    adata = kb_utils.import_matrix_as_anndata(X, obs_path, var_path)

    # Add sample name
    adata.obs["sample"] = sample

    # Append to adata list
    adatas.append(adata)

/home/laura/anaconda3/lib/python3.9/site-packages/kb_python/utils.py:673: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  anndata.AnnData(X=mtx.tocsr(), obs=df_barcodes, var=df_genes)
/home/laura/anaconda3/lib/python3.9/site-packages/kb_python/utils.py:673: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  anndata.AnnData(X=mtx.tocsr(), obs=df_barcodes, var=df_genes)
/home/laura/anaconda3/lib/python3.9/site-packages/kb_python/utils.py:673: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype exp

In [6]:
# adatas = []
# for fastq in fastqs:
#     try:
#         # Load data
#         sample = fastq.split("_L001_R1_001.fastq.gz")[0]
#         results_dir = f"{fastq_folder}/{sample}"
#         # results_dir = f"{fastq_folder}/results/{sample}"

#         # Filepath to counts
#         X = f"{results_dir}/bustools_count/output.mtx"
#         # Filepath to barcode metadata
#         var_path = f"{results_dir}/bustools_count/output.genes.txt"
#         # Filepath to gene metadata
#         obs_path = f"{results_dir}/bustools_count/output.barcodes.txt" 

#         # Create AnnData object
#         adata = kb_utils.import_matrix_as_anndata(X, obs_path, var_path)

#         # Add sample name
#         adata.obs["sample"] = sample

#         # Append to adata list
#         adatas.append(adata)
#     except:
#         continue

In [7]:
len(adatas)

88

In [8]:
# Concatenate adata objects
adata = adatas[0]
for adata_obj in adatas[1:]:
    adata = adata.concatenate(adata_obj, 
                             join='outer', 
                             batch_key='sample_index',
                             batch_categories=None, 
                             uns_merge=None, 
                             fill_value=np.nan)
    
adata

/home/laura/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/home/laura/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/home/laura/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion 

AnnData object with n_obs × n_vars = 88 × 99228
    obs: 'sample', 'sample_index'

In [9]:
# Set sample as index and drop unnecessary columns
adata.obs = adata.obs.set_index("sample")
adata.obs = adata.obs.drop("sample_index", axis=1)
adata.obs

""
sample
M006457-VSP-CIT-1-Rep1_S1_L001_R1_001
M006457-VSP-CIT-1-Rep1_S1_L001_R2_001
M006457-VSP-CIT-1-Rep2_S2_L001_R1_001
M006457-VSP-CIT-1-Rep2_S2_L001_R2_001
M006457-VSP-CIT-10-Rep1_S9_L001_R1_001
...
M006457-VSP-UHR1ng-10KCVD-Rep1_S11_L001_R2_001
M006457-VSP-UHR1ng-10KCVD-Rep2_S12_L001_R1_001
M006457-VSP-UHR1ng-10KCVD-Rep2_S12_L001_R2_001


In [10]:
clean_labels = []
for label in adata.obs.index:
    label1 = label.split("M006457-VSP-")[1].replace("CIT-", "").split("_S")[0]
    label2 = label.split("_S")[1].split("_L001")[1].replace("R1_001", "R1").replace("R2_001", "R2")
    clean_labels.append("".join([label1, label2]))
    
adata.obs["clean_label"] = clean_labels

In [11]:
clean_labels

['1-Rep1_R1',
 '1-Rep1_R2',
 '1-Rep2_R1',
 '1-Rep2_R2',
 '10-Rep1_R1',
 '10-Rep1_R2',
 '10-Rep2_R1',
 '10-Rep2_R2',
 '11-Rep1_R1',
 '11-Rep1_R2',
 '11-Rep2_R1',
 '11-Rep2_R2',
 '12-Rep1_R1',
 '12-Rep1_R2',
 '12-Rep2_R1',
 '12-Rep2_R2',
 '13-Rep1_R1',
 '13-Rep1_R2',
 '13-Rep2_R1',
 '13-Rep2_R2',
 '14-Rep1_R1',
 '14-Rep1_R2',
 '14-Rep2_R1',
 '14-Rep2_R2',
 '15-Rep1_R1',
 '15-Rep1_R2',
 '15-Rep2_R1',
 '15-Rep2_R2',
 '16-Rep1_R1',
 '16-Rep1_R2',
 '16-Rep2_R1',
 '16-Rep2_R2',
 '2-Rep1_R1',
 '2-Rep1_R2',
 '2-Rep2_R1',
 '2-Rep2_R2',
 '3-Rep1_R1',
 '3-Rep1_R2',
 '3-Rep2_R1',
 '3-Rep2_R2',
 '4-Rep1_R1',
 '4-Rep1_R2',
 '4-Rep2_R1',
 '4-Rep2_R2',
 '5-Rep1_R1',
 '5-Rep1_R2',
 '5-Rep2_R1',
 '5-Rep2_R2',
 '6-Rep1_R1',
 '6-Rep1_R2',
 '6-Rep2_R1',
 '6-Rep2_R2',
 '7-Rep1_R1',
 '7-Rep1_R2',
 '7-Rep2_R1',
 '7-Rep2_R2',
 '8-Rep1_R1',
 '8-Rep1_R2',
 '9-Rep1_R1',
 '9-Rep1_R2',
 '9-Rep2_R1',
 '9-Rep2_R2',
 'HBR-1ng-Rep2_R1',
 'HBR-1ng-Rep2_R2',
 'HBR10ng-10KCVD-Rep1_R1',
 'HBR10ng-10KCVD-Rep1_R2',
 'HBR10ng-

In [12]:
adata.write(f"{fastq_folder}/results_cdna_dna/all_VSP.h5ad")

Load metadata:

In [13]:
metadata = pd.read_excel("20230224_Metadata_ForLaura.xlsx")

In [14]:
metadata = metadata[metadata["Sequencing Method"] == "Viral Surveilance Panel Enriched"]
metadata.head()

,FASTQ Label,Sample,Sequencing Method,Specimen Type,Collection Method,Participant ID,SARS-CoV-2 Infected,Infection Stage,Final RNA Concentration (ng/uL),RIN Score,SARS-CoV-2 N1 Ct Value,SARS-CoV-2 Viral Load (copies/mL specimen),Human RNaseP Ct Value
12,AW_SB4_1,1,Viral Surveilance Panel Enriched,Saliva,Spectrum,P099,0,Pre-Infection,63.8,NaN,ND,<1000,22.340427
13,AW_SB4_2,2,Viral Surveilance Panel Enriched,Saliva,Spectrum,P099,0,Pre-Infection,106,NaN,ND,<1000,24.672273
14,AW_SB4_3,3,Viral Surveilance Panel Enriched,Saliva,Spectrum,P099,1,Peak Infection,32.2,NaN,29.121141,252501.973186,33.650029
15,AW_SB4_4,4,Viral Surveilance Panel Enriched,Saliva,Spectrum,P085,0,Uninfected Control,44.8,NaN,ND,<1000,24.602778
16,AW_SB4_5,5,Viral Surveilance Panel Enriched,Saliva,Spectrum,P165,0,Uninfected Control,45.2,NaN,ND,<1000,26.450494


Load phylogeny mapping:

In [16]:
## Get virus ID -> phylogeny mapping from tsv
# Get corresponding phylogenies for each virus ID from look-up file
phylogeny_data = pd.read_csv(u_tax_tsv, sep='\t')
phylogeny_data.head()

NameError: name 'u_tax_tsv' is not defined

In [ ]:
phylogeny_data[phylogeny_data["species"].str.contains("Severe acute respiratory syndrome")]

In [ ]:
# # Print total coronavirus counts across all samples
# target_ids = phylogeny_data[phylogeny_data["species"].str.contains("Severe acute respiratory syndrome")]["Label"].values
# for id_ in target_ids:
#     counts = adata.X[:, adata.var.index == id_].sum()
#     if counts > 0:
#         print(id_, ": ", counts)

Plot total coronavirus counts per sample:

In [15]:
adata.obs["clean_label"].values

array(['1-Rep1_R1', '1-Rep1_R2', '1-Rep2_R1', '1-Rep2_R2', '10-Rep1_R1',
       '10-Rep1_R2', '10-Rep2_R1', '10-Rep2_R2', '11-Rep1_R1',
       '11-Rep1_R2', '11-Rep2_R1', '11-Rep2_R2', '12-Rep1_R1',
       '12-Rep1_R2', '12-Rep2_R1', '12-Rep2_R2', '13-Rep1_R1',
       '13-Rep1_R2', '13-Rep2_R1', '13-Rep2_R2', '14-Rep1_R1',
       '14-Rep1_R2', '14-Rep2_R1', '14-Rep2_R2', '15-Rep1_R1',
       '15-Rep1_R2', '15-Rep2_R1', '15-Rep2_R2', '16-Rep1_R1',
       '16-Rep1_R2', '16-Rep2_R1', '16-Rep2_R2', '2-Rep1_R1', '2-Rep1_R2',
       '2-Rep2_R1', '2-Rep2_R2', '3-Rep1_R1', '3-Rep1_R2', '3-Rep2_R1',
       '3-Rep2_R2', '4-Rep1_R1', '4-Rep1_R2', '4-Rep2_R1', '4-Rep2_R2',
       '5-Rep1_R1', '5-Rep1_R2', '5-Rep2_R1', '5-Rep2_R2', '6-Rep1_R1',
       '6-Rep1_R2', '6-Rep2_R1', '6-Rep2_R2', '7-Rep1_R1', '7-Rep1_R2',
       '7-Rep2_R1', '7-Rep2_R2', '8-Rep1_R1', '8-Rep1_R2', '9-Rep1_R1',
       '9-Rep1_R2', '9-Rep2_R1', '9-Rep2_R2', 'HBR-1ng-Rep2_R1',
       'HBR-1ng-Rep2_R2', 'HBR10ng-10KCVD-Rep1_R1

In [16]:
samples = [
    '3-Rep1_R1', '3-Rep1_R2', '3-Rep2_R1', '3-Rep2_R2', 
    '9-Rep1_R1', '9-Rep1_R2', '9-Rep2_R1', '9-Rep2_R2', 
    '12-Rep1_R1', '12-Rep1_R2', '12-Rep2_R1', '12-Rep2_R2', 
    '15-Rep1_R1', '15-Rep1_R2', '15-Rep2_R1', '15-Rep2_R2', 
    '16-Rep1_R1', '16-Rep1_R2', '16-Rep2_R1', '16-Rep2_R2',
    
    '1-Rep1_R1', '1-Rep1_R2', '1-Rep2_R1', '1-Rep2_R2', 
    '2-Rep1_R1', '2-Rep1_R2', '2-Rep2_R1', '2-Rep2_R2',
    '4-Rep1_R1', '4-Rep1_R2', '4-Rep2_R1', '4-Rep2_R2',
    '5-Rep1_R1', '5-Rep1_R2', '5-Rep2_R1', '5-Rep2_R2', 
    '6-Rep1_R1', '6-Rep1_R2', '6-Rep2_R1', '6-Rep2_R2', 
    '7-Rep1_R1', '7-Rep1_R2', '7-Rep2_R1', '7-Rep2_R2', 
    '8-Rep1_R1', '8-Rep1_R2',
    '10-Rep1_R1', '10-Rep1_R2', '10-Rep2_R1', '10-Rep2_R2',
    
    '11-Rep1_R1', '11-Rep1_R2', '11-Rep2_R1', '11-Rep2_R2', 
    
    '13-Rep1_R1', '13-Rep1_R2', '13-Rep2_R1', '13-Rep2_R2', 
    '14-Rep1_R1', '14-Rep1_R2', '14-Rep2_R1', '14-Rep2_R2',
    
]

# SARS-CoV-2 Viral Load (copies/mL specimen)
viral_load = [
    2.53E+05, 2.53E+05, 2.53E+05, 2.53E+05,
    5.23E+06, 5.23E+06, 5.23E+06, 5.23E+06,
    1.10E+10, 1.10E+10, 1.10E+10, 1.10E+10,
    4.64E+06, 4.64E+06, 4.64E+06, 4.64E+06,
    1.69E+06, 1.69E+06, 1.69E+06, 1.69E+06,
    
    0, 0, 0, 0,
    0, 0, 0, 0,
    0, 0, 0, 0,
    0, 0, 0, 0,
    0, 0, 0, 0,
    0, 0, 0, 0,
    0, 0,
    0, 0, 0, 0,
    
    0, 0, 0, 0,
    
    0, 0, 0, 0,
    0, 0, 0, 0,
    # 1.76E+06, 1.76E+06, 1.76E+06, 1.76E+06 # wrong label
]

# SARS-CoV2 infected
pos = [
    True, True, True, True,
    True, True, True, True,
    True, True, True, True,
    True, True, True, True,
    True, True, True, True,
    
    False, False, False, False,
    False, False, False, False,
    False, False, False, False,
    False, False, False, False,
    False, False, False, False,
    False, False, False, False,
    False, False, 
    False, False, False, False,
    
    False, False, False, False,
    
    False, False, False, False,
    False, False, False, False,
]

# All non-Spectrum are Zymo collection method
sample_type = [
    "saliva_spectrum", "saliva_spectrum", "saliva_spectrum", "saliva_spectrum",
    "saliva", "saliva", "saliva", "saliva",
    "nasal swab", "nasal swab", "nasal swab", "nasal swab", 
    "throat swab", "throat swab", "throat swab", "throat swab", 
    "throat swab", "throat swab", "throat swab", "throat swab", 
    
    "saliva_spectrum", "saliva_spectrum", "saliva_spectrum", "saliva_spectrum",
    "saliva_spectrum", "saliva_spectrum", "saliva_spectrum", "saliva_spectrum",
    "saliva_spectrum", "saliva_spectrum", "saliva_spectrum", "saliva_spectrum",
    "saliva_spectrum", "saliva_spectrum", "saliva_spectrum", "saliva_spectrum",
    "saliva", "saliva", "saliva", "saliva",
    "saliva", "saliva", "saliva", "saliva",
    "saliva", "saliva", "saliva", "saliva",
    "saliva", "saliva", "saliva", "saliva",
    
    "nasal swab", "nasal swab", "nasal swab", "nasal swab", 
    
    "throat swab", "throat swab", "throat swab", "throat swab", 
    "throat swab", "throat swab", "throat swab", "throat swab", 
]

throat/nasal swab 200 uL extracted -> eluded in 100 uL -> 8 uL into library prep -> X uL into enrichment   
saliva 400uL extracted -> eluded in 100 uL -> 8 uL into library prep -> X uL into enrichment 

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))

target_ids = phylogeny_data[phylogeny_data["species"].str.contains("Severe acute respiratory syndrome-related coronavirus")]["Label"].values
# target_ids = phylogeny_data[phylogeny_data["genus"]=="Betacoronavirus"]["Label"].values
labels = samples

counts1 = []
# counts2 = []
for sample in samples:
    counts1.append(adata.X[adata.obs.clean_label == sample, adata.var.index.isin(target_ids)].sum())
    # counts2.append(nd(adata.X[adata.obs.clean_label == sample, adata.var.index == "u163957"])[0])

fontsize=12
width = 0.2
x = np.arange(len(labels))

ax.bar(x-0.5*width, counts1, width=width, label="PalmDB coronaviruses", edgecolor="black")
# ax.bar(x-0.5*width, counts2, width=width, label="PalmDB ID: u163957", edgecolor="black")

ax.bar(x+0.5*width, viral_load, width=width, label="RT-qPCR (copies/mL)", edgecolor="black", color="white", hatch="/")

ax.set_yscale("log")
ax.set_ylabel("Counts", fontsize=fontsize)
ax.legend(fontsize=fontsize)

ax.set_xticks(x, samples, rotation=45, ha="right")
# Mark positive samples 
for i, sample in enumerate(samples):
    if pos[i]:
        plt.gca().get_xticklabels()[i].set_color("red") 
        
ax.tick_params(axis="both", labelsize=fontsize)
ax.set_title(f"Total severe acute respiratory syndrome-related coronaviruses read counts", fontsize=fontsize+2)
    
# plt.tight_layout()

plt.savefig("figures/cov_counts_VSP_full_index.png", dpi=300, bbox_inches="tight")

fig.show()

# !!! Where were these patients in their infection timeline? Is RdRP more/less abundant in different stages of infection?

Plot fraction of viral reads out of all processed reads:

In [ ]:
total_reads = []
virus_reads = []
for sample in samples:
    # Get number of reads processed
    results_dir = f"{fastq_folder}/results/{adata.obs.index[adata.obs.clean_label == sample].values[0]}"
    with open(f'{results_dir}/run_info.json') as f:
        data = f.read()
    run_info_json = json.loads(data)
    f.close()
    total_reads.append(run_info_json["n_processed"])
    
    # Get total virus reads
    virus_reads.append(adata.X[adata.obs.clean_label == sample, :].sum())

fig, ax = plt.subplots(figsize=(20, 5))
fontsize=12
width = 0.2
labels = samples
x = np.arange(len(labels))

ax.bar(x, total_reads, width=width, label="Total reads processed", edgecolor="black")
ax.bar(x, virus_reads, width=width, label="Viral reads", edgecolor="black")

# ax.set_yscale("log")
ax.set_ylabel("Counts", fontsize=fontsize)
ax.legend(fontsize=fontsize)

ax.set_xticks(x, samples, rotation=45, ha="right")
# Mark positive samples
for i, sample in enumerate(samples):
    if pos[i]:
        plt.gca().get_xticklabels()[i].set_color("red") 
        
ax.tick_params(axis="both", labelsize=fontsize)
ax.set_title(f"Number of viral reads out of all reads processed", fontsize=fontsize+2)
    
# plt.tight_layout()

fig.show()

In [ ]:
total_df = pd.DataFrame()
total_df["sample"] = samples
total_df["total_reads"] = total_reads
total_df["virus_reads"] = virus_reads

In [ ]:
total_df["virus_fraction"] = total_df["virus_reads"] / total_df["total_reads"] 
total_df

These fractions are way too small to plot them like this.

Plot virus species fractions per sample:

In [ ]:
# Minimum total counts across all samples 
count_threshold = 100

In [ ]:
# Get counts for each virus per sample
vir_fractions_df = pd.DataFrame()
vir_fractions_df["sample"] = samples

for virus_id in adata.var.index:
    if adata.X[:, adata.var.index == virus_id].max() >= count_threshold:
        virus_counts = []
        for sample in samples:
            virus_counts.append(nd(adata.X[adata.obs.clean_label == sample, adata.var.index == virus_id])[0])
            
        utax_label = "_".join(phylogeny_data[phylogeny_data["Label"] == virus_id].values[0]).replace("_.", "")
        vir_fractions_df[utax_label] = virus_counts

# De-fragment dataframe
vir_fractions_df = vir_fractions_df.copy()

# Set sample as index
vir_fractions_df = vir_fractions_df.set_index("sample")

vir_fractions_df.head()

In [ ]:
# Get total virus reads per sample
vir_fractions_df["sample_total"] = vir_fractions_df.sum(axis = 1)

# Get fractions for each virus
for virus in vir_fractions_df.columns[:-1]:
    vir_fractions_df[f"{virus}_fraction"] = vir_fractions_df[virus] / vir_fractions_df["sample_total"]
    
# De-fragment dataframe
vir_fractions_df = vir_fractions_df.copy()

vir_fractions_df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
fontsize=12
width = 0.55
x = np.arange(len(samples))

bottom = np.zeros(len(samples))
color_idx = 0
hatch=""
for virus_column in vir_fractions_df.columns:
    if "_fraction" in virus_column:
        label_temp = virus_column.split("_fraction")[0]
        # Split long labels after n characters
        n = 79
        label = ("\n".join(label_temp[i:i+n] for i in range(0, len(label_temp), n))) if len(label_temp) > n else label_temp
        
        ax.bar(x, vir_fractions_df[virus_column].values, width, label=label, bottom=bottom, color=plt.cm.tab20(color_idx), hatch=hatch)
        bottom += vir_fractions_df[virus_column].values
        if color_idx < 19:
            color_idx+=1
        else:
            color_idx=0
            hatch="/"

# Add number of total virus counts to plot and generate xtick labels
ax.set_ylim(0, 1.1)
sample_type_label = []
y_height  = 1.01
for i, sample in enumerate(samples):
    total_count = "{:,}".format(vir_fractions_df[vir_fractions_df.index == sample]["sample_total"].values[0].astype(int))  
    ax.annotate(total_count, (i, y_height), fontsize=fontsize-2, ha="center")
    
    if y_height == 1.01:
        y_height = 1.04
    elif y_height == 1.04:
        y_height = 1.07
    else:
        y_height = 1.01
        
    sample_type_label.append(sample+"_"+sample_type[i])

# ax.set_yscale("log")
ax.set_ylabel("Fraction of total virus reads", fontsize=fontsize)
ax.legend(fontsize=fontsize, loc='center left', bbox_to_anchor=(1, 0.5))
    
ax.set_xticks(x, sample_type_label, rotation=45, ha="right")
# Mark positive samples
for i, sample in enumerate(samples):
    if pos[i]:
        plt.gca().get_xticklabels()[i].set_color("red") 
        
ax.tick_params(axis="both", labelsize=fontsize)
ax.set_title(f"Fraction of reads per virus ID", fontsize=fontsize+2)
    
ax.margins(x=0.013) 

# plt.tight_layout()

plt.savefig("figures/bar_VSP.png", dpi=300, bbox_inches="tight")

fig.show()

Same but horizontal barplot:

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
fontsize = 16
width = 0.6
x = np.arange(len(samples))

bottom = np.zeros(len(samples))
color_idx = 0
hatch=""
for virus_column in vir_fractions_df.columns:
    if "_fraction" in virus_column:
        label_temp = virus_column.split("_fraction")[0]
        # Split long labels after n characters
        n = 79
        label = ("\n".join(label_temp[i:i+n] for i in range(0, len(label_temp), n))) if len(label_temp) > n else label_temp
        
        ax.barh(x, vir_fractions_df[virus_column].values, width, label=label, left=bottom, color=plt.cm.tab20(color_idx), hatch=hatch)
        bottom += vir_fractions_df[virus_column].values
        if color_idx < 19:
            color_idx+=1
        else:
            color_idx=0
            hatch="/"

# Add number of total virus counts to plot and generate xtick labels
ax.set_xlim(0, 1.07)
sample_type_label = []
x_height  = 1.005
for i, sample in enumerate(samples):
    total_count = "{:,}".format(vir_fractions_df[vir_fractions_df.index == sample]["sample_total"].values[0].astype(int))  
    ax.annotate(total_count, (x_height, i), fontsize=fontsize, ha="left", va="center")
        
    sample_type_label.append(sample+"_"+sample_type[i])

# ax.set_yscale("log")
ax.set_xlabel("Fraction of virus counts", fontsize=fontsize+5)
ax.legend(fontsize=fontsize, loc="center left", bbox_to_anchor=(1, 0.5))
    
ax.set_yticks(x, sample_type_label, rotation=0, ha="right")
# Mark positive samples
for i, sample in enumerate(samples):
    if pos[i]:
        plt.gca().get_yticklabels()[i].set_color("red") 
        
ax.tick_params(axis="both", labelsize=fontsize)
ax.set_title(f"Fraction of counts per virus", fontsize=fontsize+10)
    
ax.margins(y=0.005) 

# plt.tight_layout()

plt.savefig("figures/barh_VSP.png", dpi=300, bbox_inches="tight")

fig.show()

Create Krona plot including sample of origin:

In [ ]:
count_threshold = 100

master = pd.DataFrame()
for i, sub_adata in enumerate(adatas):
    if "HBR" not in sub_adata.obs["sample"].values[0] and "UHR" not in sub_adata.obs["sample"].values[0]:
        # Add total number of counts (across all cells) for each virus ID to phylogeny data temp
        virus_ids = sub_adata.var.index.values
        total_counts = nd(sub_adata.X.sum(axis=0))
        total_count_dict = {virus_ids[i]: total_counts[i] for i in range(len(virus_ids))}
        phylogeny_data_temp = phylogeny_data.copy()
        phylogeny_data_temp['total_count'] = phylogeny_data_temp['Label'].map(total_count_dict)

        # Remove all viruses that were only detected in less than num_cells_threshold cells
        phylogeny_data_temp = phylogeny_data_temp[phylogeny_data_temp["total_count"] >= count_threshold]

        # Remove non-relevent columns and change order of columns  
        phylogeny_data_temp = phylogeny_data_temp[["total_count", "phylum", "class", "order", "family", "genus", "species", "Label"]]

        # Replace dots with NaN
        phylogeny_data_temp = phylogeny_data_temp.replace(".", np.nan)

        # Add column with clean sample ID
        label = sub_adata.obs["sample"].values[0]
        label1 = label.split("M006457-VSP-")[1].replace("CIT-", "").split("_S")[0]
        label2 = label.split("_S")[1].split("_L001")[1].replace("R1_001", "R1").replace("R2_001", "R2")
        phylogeny_data_temp["Sample"] = "".join([label1, label2])

        # Append to master dataframe
        if i == 0:
            master = phylogeny_data_temp.copy()
        else:
            master = master.append(phylogeny_data_temp)

# Save counts + taxnomomies data to txt
master.to_csv(f'krona_VSP.txt', sep ='\t', header=None, index=False)

# Generate Krona plot
krona_out = "figures/krona_VSP.html"
!$ktImportText krona_VSP.txt -o $krona_out -n "Viral RNAs"

___

Same for [Universal Human Reference (UHR)](https://www.agilent.com/cs/library/usermanuals/public/740000.pdf) and [Human Brain Reference (HBR)](https://www.thermofisher.com/order/catalog/product/QS0611) data (some with spike-ins):

In [ ]:
adata.obs[(adata.obs["clean_label"].str.contains("UHR")) | (adata.obs["clean_label"].str.contains("HBR"))]["clean_label"].values

In [ ]:
samples_hr = [
    'HBR1ng-Rep1_R1', 'HBR1ng-Rep1_R2',
    'HBR-1ng-Rep2_R1', 'HBR-1ng-Rep2_R2',
    'HBR10ng-Rep1_R1', 'HBR10ng-Rep1_R2',
    'HBR1ng-10KCVD-Rep1_R1', 'HBR1ng-10KCVD-Rep1_R2',
    'HBR1ng-10KCVD-Rep2_R1', 'HBR1ng-10KCVD-Rep2_R2',
    'HBR10ng-10KCVD-Rep1_R1', 'HBR10ng-10KCVD-Rep1_R2', 
    'HBR10ng-10KCVD-Rep2_R1', 'HBR10ng-10KCVD-Rep2_R2', 
    
    'UHR1ng-Rep1_R1', 'UHR1ng-Rep1_R2',
    'UHR-1ng-Rep2_R1', 'UHR-1ng-Rep2_R2',
    'UHR1ng-10KCVD-Rep1_R1', 'UHR1ng-10KCVD-Rep1_R2',
    'UHR1ng-10KCVD-Rep2_R1', 'UHR1ng-10KCVD-Rep2_R2',
    'UHR10ng-10KCVD-Rep1_R1', 'UHR10ng-10KCVD-Rep1_R2',
    'UHR10ng-10KCVD-Rep2_R1', 'UHR10ng-10KCVD-Rep2_R2',
]

viral_load_hr = []
for sample in samples_hr:
    if "10KCVD" in sample:
        viral_load_hr.append(10000)
    else:
        viral_load_hr.append(0)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))

labels = samples_hr

counts1 = []
counts2 = []
for sample in samples_hr:
    counts1.append(nd(adata.X[adata.obs.clean_label == sample, adata.var.index == "u24667"])[0])
    counts2.append(nd(adata.X[adata.obs.clean_label == sample, adata.var.index == "u163957"])[0])

fontsize=12
width = 0.2
x = np.arange(len(labels))

ax.bar(x-1.5*width, counts1, width=width, label="PalmDB ID: u24667", edgecolor="black")
ax.bar(x-0.5*width, counts2, width=width, label="PalmDB ID: u163957", edgecolor="black")

ax.bar(x+0.5*width, viral_load_hr, width=width, label="SARS-CoV2 spike-in", edgecolor="black", color="white", hatch="/")

ax.set_yscale("log")
ax.set_ylabel("Counts", fontsize=fontsize)
ax.legend(fontsize=fontsize)

ax.set_xticks(x, samples_hr, rotation=45, ha="right")
# Mark positive samples 
for i, sample in enumerate(samples_hr):
    if "10KCVD" in sample:
        plt.gca().get_xticklabels()[i].set_color("red") 
        
ax.tick_params(axis="both", labelsize=fontsize)
ax.set_title(f"Total Coronaviridae read counts", fontsize=fontsize+2)
    
# plt.tight_layout()

plt.savefig("figures/cov_counts_hr.png", dpi=300, bbox_inches="tight")

fig.show()

Positive counts in non-spiked cells due to library cross contamination?

In [ ]:
# Minimum total counts across all samples 
count_threshold = 100

# Get counts for each virus per sample
vir_fractions_df_hr = pd.DataFrame()
vir_fractions_df_hr["sample"] = samples_hr

for virus_id in adata.var.index:
    if adata.X[:, adata.var.index == virus_id].max() >= count_threshold:
        virus_counts = []
        for sample in samples_hr:
            virus_counts.append(nd(adata.X[adata.obs.clean_label == sample, adata.var.index == virus_id])[0])
            
        utax_label = "_".join(phylogeny_data[phylogeny_data["Label"] == virus_id].values[0]).replace("_.", "")
        vir_fractions_df_hr[utax_label] = virus_counts

# De-fragment dataframe
vir_fractions_df_hr = vir_fractions_df_hr.copy()

# Set sample as index
vir_fractions_df_hr = vir_fractions_df_hr.set_index("sample")

vir_fractions_df_hr.head()

In [ ]:
# Get total virus reads per sample
vir_fractions_df_hr["sample_total"] = vir_fractions_df_hr.sum(axis = 1)

# Get fractions for each virus
for virus in vir_fractions_df_hr.columns[:-1]:
    vir_fractions_df_hr[f"{virus}_fraction"] = vir_fractions_df_hr[virus] / vir_fractions_df_hr["sample_total"]
    
# De-fragment dataframe
vir_fractions_df_hr = vir_fractions_df_hr.copy()

vir_fractions_df_hr.head()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
fontsize = 16
width = 0.6
x = np.arange(len(samples_hr))

bottom = np.zeros(len(samples_hr))
color_idx = 0
hatch=""
for virus_column in vir_fractions_df_hr.columns:
    if "_fraction" in virus_column:
        label_temp = virus_column.split("_fraction")[0]
        # Split long labels after n characters
        n = 79
        label = ("\n".join(label_temp[i:i+n] for i in range(0, len(label_temp), n))) if len(label_temp) > n else label_temp
        
        ax.barh(x, vir_fractions_df_hr[virus_column].values, width, label=label, left=bottom, color=plt.cm.tab20(color_idx), hatch=hatch)
        bottom += vir_fractions_df_hr[virus_column].values
        if color_idx <= 20:
            color_idx+=1
        else:
            color_idx=0
            hatch="/"

# Add number of total virus counts to plot and generate xtick labels
ax.set_xlim(0, 1.075)
sample_type_label = []
x_height  = 1.005
for i, sample in enumerate(samples_hr):
    total_count = "{:,}".format(vir_fractions_df_hr[vir_fractions_df_hr.index == sample]["sample_total"].values[0].astype(int))  
    ax.annotate(total_count, (x_height, i), fontsize=fontsize, ha="left", va="center")
        
    sample_type_label.append(sample+"_"+sample_type[i])

# ax.set_yscale("log")
ax.set_xlabel("Fraction of total virus counts", fontsize=fontsize+5)
ax.legend(fontsize=fontsize, loc="center left", bbox_to_anchor=(1, 0.5))
    
ax.set_yticks(x, sample_type_label, rotation=0, ha="right")
# Mark positive samples
for i, sample in enumerate(samples_hr):
    if "10KCVD" in sample:
        plt.gca().get_yticklabels()[i].set_color("red") 
        
ax.tick_params(axis="both", labelsize=fontsize+5)
ax.set_title(f"Fraction of counts per virus", fontsize=fontsize+10)
    
ax.margins(y=0.005) 

# plt.tight_layout()

plt.savefig("figures/barh_hr.png", dpi=300, bbox_inches="tight")

fig.show()

Kronaplot:

In [ ]:
count_threshold = 100

master = pd.DataFrame()
for i, sub_adata in enumerate(adatas):
    if "HBR" in sub_adata.obs["sample"].values[0] or "UHR" in sub_adata.obs["sample"].values[0]:
        # Add total number of counts (across all cells) for each virus ID to phylogeny data temp
        virus_ids = sub_adata.var.index.values
        total_counts = nd(sub_adata.X.sum(axis=0))
        total_count_dict = {virus_ids[i]: total_counts[i] for i in range(len(virus_ids))}
        phylogeny_data_temp = phylogeny_data.copy()
        phylogeny_data_temp['total_count'] = phylogeny_data_temp['Label'].map(total_count_dict)

        # Remove all viruses that were only detected in less than num_cells_threshold cells
        phylogeny_data_temp = phylogeny_data_temp[phylogeny_data_temp["total_count"] >= count_threshold]

        # Remove non-relevent columns and change order of columns  
        phylogeny_data_temp = phylogeny_data_temp[["total_count", "phylum", "class", "order", "family", "genus", "species", "Label"]]

        # Replace dots with NaN
        phylogeny_data_temp = phylogeny_data_temp.replace(".", np.nan)

        # Add column with clean sample ID
        label = sub_adata.obs["sample"].values[0]
        label1 = label.split("M006457-VSP-")[1].replace("CIT-", "").split("_S")[0]
        label2 = label.split("_S")[1].split("_L001")[1].replace("R1_001", "R1").replace("R2_001", "R2")
        phylogeny_data_temp["Sample"] = "".join([label1, label2])

        # Append to master dataframe
        if i == 0:
            master = phylogeny_data_temp.copy()
        else:
            master = master.append(phylogeny_data_temp)

# Save counts + taxnomomies data to txt
master.to_csv(f'krona_hr.txt', sep ='\t', header=None, index=False)

# Generate Krona plot
krona_out = "figures/krona_hr.html"
!$ktImportText krona_hr.txt -o $krona_out -n "Viral RNAs"